In [150]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from natsort import natsorted, ns
from functools import reduce

In [151]:
# Reading in my format
# root_paths = ['/home/eduardo/Desktop/new_datasets/ATP',
#               '/home/eduardo/Desktop/new_datasets/Earthquake',
#               '/home/eduardo/Desktop/new_datasets/WB',
#               '/home/eduardo/Desktop/new_datasets/TMDB',
#               '/home/eduardo/Desktop/new_datasets/GH']

# dataset_paths = []
# for rp in root_paths:
#     paths = [x[0] for x in os.walk(rp) if x[0] != rp]
#     dataset_paths.extend(paths)
    
# dataset_paths[:6]

In [152]:
root_paths = [ '/home/eduardo/Desktop/new_datasets/converted/converted_1',
                 '/home/eduardo/Desktop/new_datasets/converted/converted_5',
                 '/home/eduardo/Desktop/new_datasets/converted/converted_3',
                 '/home/eduardo/Desktop/new_datasets/converted/converted_2',
                 '/home/eduardo/Desktop/new_datasets/converted/converted_4',
                 '/home/eduardo/Desktop/new_datasets/OLD']

dataset_paths = []
for rp in root_paths:
    onlyfiles = [os.path.join(rp,f) for f in os.listdir(rp) if os.path.isfile(os.path.join(rp, f))]
    dataset_paths.extend(onlyfiles)

print(len(dataset_paths))    
dataset_paths[:6]

1506


['/home/eduardo/Desktop/new_datasets/converted/converted_1/wb-SL.AGR.EMPL.FE.ZS.data',
 '/home/eduardo/Desktop/new_datasets/converted/converted_1/wb-SL.EMP.MPYR.FE.ZS.data',
 '/home/eduardo/Desktop/new_datasets/converted/converted_1/wb-SH.XPD.GHED.PC.CD.data',
 '/home/eduardo/Desktop/new_datasets/converted/converted_1/wb-TX.UVI.MRCH.XD.WD.data',
 '/home/eduardo/Desktop/new_datasets/converted/converted_1/wb-SL.GDP.PCAP.EM.KD.data',
 '/home/eduardo/Desktop/new_datasets/converted/converted_1/wb-SP.POP.0014.FE.ZS.data']

In [155]:
# Read csv
df = pd.read_csv(dataset_paths[0], header=None)

In [154]:
# Remove non-leaf rows
df = df[~df[0].isin(df[1])]
# Remove implicit parent info
df.drop(1, axis=1, inplace=True)
# Set first col as index
df.set_index(df.columns[0], inplace=True)
# Rename columns
df.columns = range(df.shape[1])
# Replace 0s with NaN
df.replace({0:np.nan}, inplace=True)
display(df)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,,,,,,,,,,,,,,,,,,,,,
root/EastAsia&Pacific/Albania,0.232000,0.283,0.366,0.393,0.428,0.411,0.312000,0.402,0.461,0.456,...,0.610,0.370,0.510,1.197,1.059,1.037,0.559,0.983,1.020,1.021
root/EastAsia&Pacific/Argentina,2.016000,2.411,3.174,2.429,2.339,2.406,2.438000,2.399,2.493,2.960,...,2.889,3.010,3.002,2.851,2.784,2.579,2.344,2.371,2.438,2.374
root/EastAsia&Pacific/Australia,5.446000,5.285,5.347,5.532,5.531,5.431,5.514000,5.547,5.216,5.337,...,4.967,4.634,4.775,4.610,4.396,4.355,4.322,4.372,4.244,4.198
root/EastAsia&Pacific/Bolivia,2.625000,2.637,2.703,2.741,2.662,2.708,2.804000,2.807,2.913,2.845,...,2.409,2.520,2.466,2.389,2.407,2.436,2.496,2.708,2.795,2.798
root/EastAsia&Pacific/CongoRep,0.461000,0.488,0.584,0.735,0.514,0.581,0.666000,0.531,0.739,0.553,...,0.624,0.553,0.544,0.676,0.660,0.684,0.598,0.663,0.794,0.794
root/EastAsia&Pacific/GuineaBissau,0.418000,0.572,0.499,0.500,0.494,0.457,0.499000,0.929,0.324,0.864,...,0.661,0.685,0.628,0.549,0.809,0.725,0.846,0.666,0.617,0.617
root/EastAsia&Pacific/Haiti,1.013000,1.079,1.046,1.095,0.894,0.805,0.855000,0.821,0.847,0.889,...,0.863,0.858,0.851,0.819,0.807,0.797,0.782,0.774,0.767,0.769
root/EastAsia&Pacific/Ireland,2.358000,2.483,2.736,2.605,2.441,2.424,2.567000,2.946,2.769,2.729,...,2.341,2.442,2.360,2.261,2.304,2.488,2.246,2.308,2.414,2.386
root/EastAsia&Pacific/Kazakhstan,0.224000,1.156,0.450,0.376,0.174,0.145,0.145000,0.170,0.275,0.487,...,1.121,1.693,1.569,1.718,1.388,1.136,1.104,0.800,0.856,0.868
